We read each csv file, preprocess it, then add them to a SQLite as a table.
- Historical prices
- Options Data
- Portfolio holdings
- News articles

For version1, we have just downloaded historical data downloaded from barchart.com

In [1]:
# Run as is
import numpy as np
import pandas as pd
# pd.set_option('display.max_rows', 25)
pd.set_option('display.max_columns', 15)

In [2]:
import os
import glob

def count_files_by_extension(path):
    extensions = {}
    for file in os.listdir(path):
        extension = os.path.splitext(file)[1]
        if extension not in extensions:
            extensions[extension] = 1
        else:
            extensions[extension] += 1
    return extensions

def allfilesinpath(path):
    """
    Get all files in a directory specified by the path.

    Args:
        path (str): The path of the directory.

    Returns:
        A list of all files in the directory.
    """
    # Create a list of all files in the specified path
    all_files = glob.glob(path + "/*")
  
    return all_files

# This function gets all the files in the path, and separates them by substrings
def separate_files_by_substrings_in_path(path, substr_list):
    # Creates a list of all files in the path
    file_paths = glob.glob(path + "/*")

    # Extract the file names and separate them by substrings
    file_data = {}
    for substr in substr_list:
        file_data[substr] = []
    for file_path in file_paths:
        file_name = os.path.basename(file_path)
        for substr in substr_list:
            if substr in file_name:
                file_data[substr].append(file_name)

    return file_data


In [3]:
path = 'Barchart-downloads/'
count_files_by_extension(path)

{'': 1, '.csv': 10}

In [4]:
file_paths = allfilesinpath(path)
file_paths
# file_names

['Barchart-downloads/spy_daily_historical-data-04-24-2023.csv',
 'Barchart-downloads/hyg_daily_historical-data-04-24-2023.csv',
 'Barchart-downloads/vnq_options-overview-history-04-24-2023.csv',
 'Barchart-downloads/tlt_daily_historical-data-04-24-2023.csv',
 'Barchart-downloads/vnq_daily_historical-data-04-24-2023.csv',
 'Barchart-downloads/vnq_options-overview-history-04-24-2023-2.csv',
 'Barchart-downloads/vnq_options-overview-history-04-24-2023-3.csv',
 'Barchart-downloads/lqd_daily_historical-data-04-24-2023.csv',
 'Barchart-downloads/spy_options-overview-history-04-24-2023 copy.csv',
 'Barchart-downloads/spy_options-overview-history-04-24-2023-2.csv']

In [5]:
# substrings = ['spy','hyg','vnq','tlt','lqd']
substrings = ['daily']
names = separate_files_by_substrings_in_path(path,substrings)
names

{'daily': ['spy_daily_historical-data-04-24-2023.csv',
  'hyg_daily_historical-data-04-24-2023.csv',
  'tlt_daily_historical-data-04-24-2023.csv',
  'vnq_daily_historical-data-04-24-2023.csv',
  'lqd_daily_historical-data-04-24-2023.csv']}

In [6]:
# Function to read the CSV file
def readcsv(file):
    # parse_dates = ['Time']
    df1 = pd.read_csv(file)
    # df1 = pd.read_csv(file, parse_dates=parse_dates)

    return df1

# Function to perform preliminary preprocessing of raw etf data downloaded from Barchart.com
def preprocess_historical_etf_data(historical_etf_data):
    # Drop last row because it has irrelevant txt
    historical_etf_data = historical_etf_data[:-1]
    
    # rename colomns
    historical_etf_data = historical_etf_data.rename(columns={"Time": "date","Change":"daily_change","%Chg":"perct_chg"})
    
    # extract percentage from %Chg string column
    historical_etf_data['perct_chg'] = historical_etf_data['perct_chg'].str.replace('%', '').astype(float)

    # convert all column headers to lower case
    historical_etf_data.columns = historical_etf_data.columns.str.lower()

    # convert the date column to a datetime object
    historical_etf_data['date'] = pd.to_datetime(historical_etf_data['date'])

    return historical_etf_data

import sqlite3

def write_dataframe_to_sqlite(df, db_file, table_name=None):
    '''
    Write a pandas dataframe to a new table in a SQLite database.
    
    Parameters:
        df (pandas.DataFrame): The dataframe to write to the database.
        db_file (str): The name of the SQLite database file.
        table_name (str): The name of the table in the database (optional).
                          If not specified, the table will be named after the
                          dataframe.
    '''
    # create a connection to the database
    conn = sqlite3.connect(db_file)

    # if table_name is not specified, use the name of the dataframe
    if table_name is None:
        table_name = df.name

    # write the dataframe to a new table in the database
    df.to_sql(name=table_name, con=conn, if_exists='replace', index=False)

    # close the database connection
    conn.close()


In [7]:
df3 = readcsv(path + "spy_daily_historical-data-04-24-2023.csv")
df3

,Time,Open,High,Low,Last,Change,%Chg,Volume
0,04/24/2023,411.9900,413.0700,410.6000,412.6300,0.4300,+0.10%,64332000.0
1,04/21/2023,412.1900,412.6800,410.1700,412.2000,0.3200,+0.08%,73457400.0
2,04/20/2023,411.2100,413.7000,410.2700,411.8800,-2.2600,-0.55%,75840300.0
3,04/19/2023,412.2200,415.0800,412.1600,414.1400,-0.0700,-0.02%,55227300.0
4,04/18/2023,415.5800,415.7200,412.7802,414.2100,0.2700,+0.07%,63559900.0
...,...,...,...,...,...,...,...,...
5860,01/06/2000,91.1541,92.3749,89.9268,89.9268,-1.4689,-1.61%,9538833.0
5861,01/05/2000,91.3565,92.3945,89.6004,91.3957,0.1632,+0.18%,18654116.0
5862,01/04/2000,93.7002,94.0462,91.1541,91.2325,-3.7146,-3.91%,12391960.0
5863,01/03/2000,96.7815,96.7815,93.9286,94.9471,-0.9400,-0.98%,12506077.0


In [22]:
# Import the required modules
import sqlite3

# Function to create sqlite_db with schema
def create_sqlite_db(names):
    sqlite_db_name = 'etf_data.db'
    
    # Create a connection to the SQLite database
    conn = sqlite3.connect(sqlite_db_name)
    
    # Create a cursor object to execute SQL queries
    cursor = conn.cursor()
    
    # Create the metadata table if it doesn't exist
    create_metadata_table = '''
    CREATE TABLE IF NOT EXISTS metadata (
        table_name TEXT,
        description TEXT,
        columns TEXT
    )
    '''
    cursor.execute(create_metadata_table)

    metadata_description = ['This table contains SPY ETF historical performance.',
                            'This table contains HYG ETF historical performance.',
                            'This table contains TLT ETF historical performance.',
                            'This table contains VNQ ETF historical performance.',
                            'This table contains LQD ETF historical performance.']

    # For all etfs
    for i, name in enumerate(names['daily']):
        
        # Read the file
        df = readcsv(path + str(name))
        
        # Do some preprocessing and get the df in our required format
        df = preprocess_historical_etf_data(df)
        
        # Write the formatted df as a table to the sqlite db
        table_name = str(name[:3])
        df.to_sql(table_name, conn, if_exists='replace', index=False)
        
        # Extract column names as metadata
        metadata_columns = ', '.join(df.columns)
        
        # Add metadata to the metadata table
        metadata_query = '''
        INSERT INTO metadata (table_name, description, columns)
        VALUES (?, ?, ?)
        '''
        metadata_values = (table_name, metadata_description[i], metadata_columns)
        cursor.execute(metadata_query, metadata_values)

    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    return sqlite_db_name


def view_table_contents(db_file, table_name):
    '''
    View the contents of a table in a SQLite database.
    
    Parameters:
        db_file (str): The name of the SQLite database file.
        table_name (str): The name of the table to view.
    '''
    # create a connection to the database
    conn = sqlite3.connect(db_file)

    # Get the first and last dates for the table
    # query = f"SELECT MIN(date), MAX(date) FROM {table_name}"
    # date_range = conn.execute(query).fetchone()
    # print(f"Date range for {table_name}: {date_range[0]} - {date_range[1]}\n")

    # read the contents of the table into a pandas dataframe
    df = pd.read_sql_query(f"SELECT * from {table_name}", conn)
    # print the contents of the dataframe
    print(df)

    # close the database connection
    conn.close()



In [23]:
sqlite_db_name = create_sqlite_db(names)

# etf_list is also the tabe name in the sqlite db
etf_list = ['spy', 'tlt', 'hyg', 'lqd', 'vnq', 'metadata']
for table_name in etf_list:
    view_table_contents('etf_data.db', table_name)

# We now have a sqlite database

                     date      open      high       low      last  \
0     2023-04-24 00:00:00  411.9900  413.0700  410.6000  412.6300   
1     2023-04-21 00:00:00  412.1900  412.6800  410.1700  412.2000   
2     2023-04-20 00:00:00  411.2100  413.7000  410.2700  411.8800   
3     2023-04-19 00:00:00  412.2200  415.0800  412.1600  414.1400   
4     2023-04-18 00:00:00  415.5800  415.7200  412.7802  414.2100   
...                   ...       ...       ...       ...       ...   
5859  2000-01-07 00:00:00   91.5981   95.1494   91.4348   95.1494   
5860  2000-01-06 00:00:00   91.1541   92.3749   89.9268   89.9268   
5861  2000-01-05 00:00:00   91.3565   92.3945   89.6004   91.3957   
5862  2000-01-04 00:00:00   93.7002   94.0462   91.1541   91.2325   
5863  2000-01-03 00:00:00   96.7815   96.7815   93.9286   94.9471   

      daily_change  perct_chg      volume  
0           0.4300       0.10  64332000.0  
1           0.3200       0.08  73457400.0  
2          -2.2600      -0.55  75840300

In [17]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
connection = sqlite3.connect('etf_data.db')

# List of table names
etf_list = ['spy', 'tlt', 'hyg', 'lqd', 'vnq']

# Loop through the list and display data for each table
for table_name in etf_list:
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", connection)
    print(f"Data for table {table_name}:")
    print(df)

# Close the connection
connection.close()

Data for table spy:
                     date      open      high       low      last  \
0     2023-04-24 00:00:00  411.9900  413.0700  410.6000  412.6300   
1     2023-04-21 00:00:00  412.1900  412.6800  410.1700  412.2000   
2     2023-04-20 00:00:00  411.2100  413.7000  410.2700  411.8800   
3     2023-04-19 00:00:00  412.2200  415.0800  412.1600  414.1400   
4     2023-04-18 00:00:00  415.5800  415.7200  412.7802  414.2100   
...                   ...       ...       ...       ...       ...   
5859  2000-01-07 00:00:00   91.5981   95.1494   91.4348   95.1494   
5860  2000-01-06 00:00:00   91.1541   92.3749   89.9268   89.9268   
5861  2000-01-05 00:00:00   91.3565   92.3945   89.6004   91.3957   
5862  2000-01-04 00:00:00   93.7002   94.0462   91.1541   91.2325   
5863  2000-01-03 00:00:00   96.7815   96.7815   93.9286   94.9471   

      daily_change  perct_chg      volume  
0           0.4300       0.10  64332000.0  
1           0.3200       0.08  73457400.0  
2          -2.2600 

In [ ]:
2023-04-24
2023-04-24
2023-04-24

In [ ]:
from os import getenv
from dotenv import load_dotenv

load_dotenv()

# Values of env variables
DB_URL = getenv("DB_URL")

# If DB exists,
if DB_URL:
    
    # Connect to db
    conn = sqlite3.connect(DB_URL)